In [27]:
## Group and filter matrices from general model to specific
## Import libraries

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
from geopandas.tools import sjoin

In [28]:
## Group filter matrix
## Group general zones based on a detailed zoning model, it also can calculate by mode
## Read input data (example 85)

path_zat = 'Proyectos/85/ZAT.shp'
path_grouped_zat = 'Proyectos/85/zat_agrupadas.shp'
path_matrices = "Proyectos/85/Matrices_2019_organizadas.xlsx"

zat = gpd.read_file(path_zat)
grouped_ZAT = gpd.read_file(path_grouped_zat)
matrices = pd.read_excel(path_matrices)

In [29]:
## Prepare data for analysis
## 01. Calculate centroids from zat

zatCentroid = zat.copy()
zatCentroid['geometry'] = zatCentroid['geometry'].centroid

## 02. Spatial join

joinedCentroid = sjoin(zatCentroid, grouped_ZAT, how = 'left')
joinedCentroid = joinedCentroid.reset_index()
joinedCentroid = joinedCentroid[joinedCentroid['index_right'].notna()]
joinedCentroid = pd.DataFrame(joinedCentroid)
## 03. Clear dataframe and delete geometry
joinedCentroid = joinedCentroid.drop(columns = {'index','geometry','index_right'})


In [30]:
## Join trips data
joinedCentroid = joinedCentroid.reset_index()
matrices_merged = pd.merge(matrices, joinedCentroid, how = "left", left_on = "De", right_on = "ZAT")
matrices_merged = matrices_merged.drop(columns=['MUNCod','NOMMun','ZAT','index','Area','UTAM'])
matrices_merged = matrices_merged.rename(columns={'id':'ZAT_agrupada_or'})

matrices_merged = pd.merge(matrices_merged, joinedCentroid, how = "left", left_on = "A", right_on = "ZAT")
matrices_merged = matrices_merged.drop(columns=['MUNCod','NOMMun','ZAT','index','Area','UTAM'])
matrices_merged = matrices_merged.rename(columns={'id':'ZAT_agrupada_des'})
print(matrices_merged)


                     Modo    De     A     Viajes  ZAT_agrupada_or  \
0        Vehiculo_privado     1    45  15.951770              NaN   
1        Vehiculo_privado     1    53  22.308970              NaN   
2        Vehiculo_privado     1    56  44.617920              NaN   
3        Vehiculo_privado     1    60  35.017940              NaN   
4        Vehiculo_privado     1    64  15.963670              NaN   
...                   ...   ...   ...        ...              ...   
46423  Transporte_publico  1844   327  20.280320              NaN   
46424  Transporte_publico  1844   912   6.116250              NaN   
46425  Transporte_publico  1845  1827   0.651951              NaN   
46426  Transporte_publico  1903   913   6.116250              NaN   
46427  Transporte_publico  1903  1844   6.116250              NaN   

       ZAT_agrupada_des  
0                   NaN  
1                   NaN  
2                   NaN  
3                   NaN  
4                   NaN  
...            

In [24]:
## Procesamiento de matrices

## Livianos con Vehiculo_privado, Taxi_ocupado, Taxi_libre y Moto
livianos = matrices_merged[(matrices_merged['Modo'] == 'Vehiculo_privado') | (matrices_merged['Modo'] == 'Taxi_libre') | (matrices_merged['Modo'] == 'Taxi_ocupado') | (matrices_merged['Modo'] == 'Motos')]
livianos_agrupad = livianos.groupby(['ZAT_agrupada_or','ZAT_agrupada_des'])['Viajes'].sum().reset_index(name="Viajes")
livianos_agrupad
# Camiones pequenos
cp = matrices_merged[matrices_merged['Modo'] == 'Camiones_pequeños']
cp = cp.groupby(['ZAT_agrupada_or','ZAT_agrupada_des'])['Viajes'].sum().reset_index(name="Viajes")
cp
# Camiones grandes
cg = matrices_merged[matrices_merged['Modo'] == 'Camiones_grandes']
cg = cg.groupby(['ZAT_agrupada_or','ZAT_agrupada_des'])['Viajes'].sum().reset_index(name="Viajes")
cg
# Motos
motos = matrices_merged[matrices_merged['Modo'] == 'Motos']
motos = motos.groupby(['ZAT_agrupada_or','ZAT_agrupada_des'])['Viajes'].sum().reset_index(name="Viajes")
livianos_agrupad

,ZAT_agrupada_or,ZAT_agrupada_des,Viajes
0,1.0,1.0,303.013430
1,1.0,2.0,230.700493
2,1.0,3.0,25.757867
3,1.0,14.0,27.977450
4,1.0,15.0,1393.189390
5,2.0,1.0,77.765598
6,2.0,2.0,46.858763
7,2.0,3.0,300.612120
8,2.0,9.0,6.924690
9,2.0,10.0,6.924690


In [25]:
livianos_agrupad

,ZAT_agrupada_or,ZAT_agrupada_des,Viajes
0,1.0,1.0,303.013430
1,1.0,2.0,230.700493
2,1.0,3.0,25.757867
3,1.0,14.0,27.977450
4,1.0,15.0,1393.189390
5,2.0,1.0,77.765598
6,2.0,2.0,46.858763
7,2.0,3.0,300.612120
8,2.0,9.0,6.924690
9,2.0,10.0,6.924690


In [26]:
## export results in excel
livianos_agrupad.to_excel('Mat_livianos_base_v02.xlsx')
cp.to_excel('Mat_camion_pequeno_base_v02.xlsx')
cg.to_excel('Mat_camion_grande_base_v02.xlsx')
motos.to_excel('Mat_motos_base.xlsx')

In [ ]:
for columna in columnas:
    Fig1.plot(T3.Wavelength, T3.columna, color = 'black'......)